In [ ]:
import numpy as np
import tensorflow as tens
import os
import shutil
import datetime
import cv2
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/data.zip -d Data

Archive:  /content/drive/MyDrive/data.zip
replace Data/train/Signed/1 (1).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#sub dir
sign_train=os.path.join('/content/Data/train/Signed')
unsign_train=os.path.join('/content/Data/train/Unsigned')
sign_val=os.path.join('/content/Data/val/Signed')
unsign_val=os.path.join('/content/Data/val/Unsigned')

In [ ]:
print('signed img in training:',len(os.listdir(sign_train)))
print('unsigned img in training:',len(os.listdir(unsign_train)))
print('signed img in val:',len(os.listdir(sign_val)))
print('unsigned img in val:',len(os.listdir(unsign_val)))

signed img in training: 23489
unsigned img in training: 23489
signed img in val: 1000
unsigned img in val: 1000


In [ ]:
train_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_gener=train_gen.flow_from_directory('/content/Data/train', classes=['Signed','Unsigned'],target_size=(256,256),
                                          batch_size=100,class_mode='binary',shuffle=True)

Found 46978 images belonging to 2 classes.


In [ ]:
val_gener = train_gen.flow_from_directory('/content/Data/val', classes=['Signed','Unsigned'],target_size=(256,256),
                                         batch_size=50,class_mode='binary',shuffle=True)

Found 2000 images belonging to 2 classes.


In [ ]:
#building the model
model = Sequential()
model.add(Conv2D(input_shape=(256,256,3),filters=60,kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=60,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=120,kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=120,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=248,kernel_size=(3,3),padding='same',activation='relu'))
model.add(Conv2D(filters=248,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))


In [ ]:
model.add(Flatten())
model.add(Dense(units=1000,activation='relu'))
model.add(Dense(units=1000,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 60)      1680      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 60)      32460     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 60)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 120)     64920     
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 120)     129720    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 120)      0         
 2D)                                                    

In [ ]:
model.compile(optimizer = tens.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history=model.fit(train_gener,validation_data=val_gener,epochs=10,verbose=1)

Epoch 1/10
470/470 [==============================] - 688s 1s/step - loss: 0.2728 - accuracy: 0.8888 - val_loss: 1.4324 - val_accuracy: 0.6195
Epoch 2/10
470/470 [==============================] - 652s 1s/step - loss: 0.1281 - accuracy: 0.9514 - val_loss: 1.8039 - val_accuracy: 0.6380
Epoch 3/10
470/470 [==============================] - 648s 1s/step - loss: 0.1004 - accuracy: 0.9609 - val_loss: 2.4696 - val_accuracy: 0.7000
Epoch 4/10
470/470 [==============================] - 648s 1s/step - loss: 0.0834 - accuracy: 0.9668 - val_loss: 3.6964 - val_accuracy: 0.6270
Epoch 5/10
470/470 [==============================] - 647s 1s/step - loss: 0.0748 - accuracy: 0.9703 - val_loss: 2.5770 - val_accuracy: 0.7355
Epoch 6/10
470/470 [==============================] - 644s 1s/step - loss: 0.0679 - accuracy: 0.9728 - val_loss: 3.9234 - val_accuracy: 0.6410
Epoch 7/10
470/470 [==============================] - 642s 1s/step - loss: 0.0604 - accuracy: 0.9755 - val_loss: 6.4493 - val_accuracy: 0.5405

In [ ]:
!unzip -q "/content/drive/MyDrive/inference.zip" -d infer

In [ ]:
infer_dataset=train_gen.flow_from_directory("/content/infer",
target_size=(256,256),batch_size=64, class_mode='binary',shuffle="True")

Found 6000 images belonging to 2 classes.


In [ ]:
model.evaluate(infer_dataset)

94/94 [==============================] - 34s 310ms/step - loss: 0.3521 - accuracy: 0.9652


[0.35210779309272766, 0.9651666879653931]

In [ ]:
model.save('/content/drive/MyDrive/MODELS.h5') 

In [ ]:
model = tens.keras.models.load_model('/content/drive/MyDrive/MODELS.h5')
model

DUMMY MODEL

**opencv & mediapipe**
